<a href="https://colab.research.google.com/github/anish-mulay07/Quantitative-Stock-Price-Analysis-GPT-Investor/blob/main/gpt_investor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install langchain
# !pip install bitsandbytes accelerate transformers
# !pip install sentence_transformers

!pip install -r "/content/drive/MyDrive/GeorgiaTech Fintech Project/LangchainDocuments-main/LangchainDocuments-main/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install streamlit

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install langchain
!pip install bitsandbytes accelerate transformers
!pip install sentence_transformers


In [ ]:
import torch
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys
import accelerate

In [ ]:
import os
# Import OpenAI as main LLM service
# from langchain.llms import OpenAI
# from langchain.embeddings import OpenAIEmbeddings
# Bring in streamlit for UI/app interface
import streamlit as st

# Import PDF document loaders...there's other ones as well!
from langchain.document_loaders import PyPDFLoader
# Import chroma as the vector store
from langchain.vectorstores import Chroma

# Import vector store stuff
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit as VST,
    VectorStoreInfo as VSI
)

In [ ]:
# # Import os to set API key
# import os
# # Import OpenAI as main LLM service
# from langchain.llms import OpenAI
# from langchain.embeddings import OpenAIEmbeddings
# # Bring in streamlit for UI/app interface
# import streamlit as st

# # Import PDF document loaders...there's other ones as well!
# from langchain.document_loaders import PyPDFLoader
# # Import chroma as the vector store
# from langchain.vectorstores import Chroma

# # Import vector store stuff
# from langchain.agents.agent_toolkits import (
#     create_vectorstore_agent,
#     VectorStoreToolkit,
#     VectorStoreInfo
# )

# Set APIkey for OpenAI Service
# Can sub this out for other LLM providers
# os.environ['OPENAI_API_KEY'] = 'youropenaiapikeyhere'

# Create instance of OpenAI LLM
# llm = OpenAI(temperature=0.1, verbose=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                            #  torch_dtype=torch.float16,
                                            #  use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  model_kwargs={"torch_dtype": torch.float16}
                                             )


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()



In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:

# # Create and load PDF Loader
# loader = PyPDFLoader('/content/drive/MyDrive/GeorgiaTech Fintech Project/LangchainDocuments-main/LangchainDocuments-main/annualreport.pdf')
# # Split pages from pdf
# pages = loader.load_and_split()

# # Load documents into vector database aka ChromaDB
# store = Chroma.from_documents(pages, embeddings, collection_name='annualreport')

# # Create vectorstore info object - metadata repo?
# vectorstore_info = VectorStoreInfo(
#     name="annual_report",
#     description="a banking annual report as a pdf",
#     vectorstore=store
# )
# # Convert the document store into a langchain toolkit
# toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

# # Add the toolkit to an end-to-end LC
# agent_executor = create_vectorstore_agent(
#     llm=llm,
#     toolkit=toolkit,
#     verbose=True
# )

In [ ]:
!pip install langchain-chroma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
# from langchain.agents import create_vectorstore_agent
# from langchain.docstore.document import VectorStoreInfo
from langchain_community.document_loaders import TextLoader
# from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
# from langchain_chroma import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
loader = PyPDFLoader("/content/drive/MyDrive/GeorgiaTech Fintech Project/LangchainDocuments-main/LangchainDocuments-main/annualreport.pdf")
pages = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(pages)




# # Load the PDF document
# loader = PyPDFLoader("/content/drive/MyDrive/GeorgiaTech Fintech Project/LangchainDocuments-main/LangchainDocuments-main/annualreport.pdf")
# pages = loader.load()

# # Create a vector store from the documents
# vectorstore = Chroma.from_documents(pages, embeddings, collection_name="annualreport")

# Create a vector store info object
# vectorstore_info = VectorStoreInfo(
#     name="annual_report",
#     description="a banking annual report as a pdf",
#     vectorstore=vectorstore
# )

# # Create a vector store agent
# agent_executor = create_vectorstore_agent(llm, vectorstore_info)

In [ ]:
db = Chroma.from_documents(documents, HuggingFaceEmbeddings())

In [ ]:
# Create a vector store info object
# from langchain.docstore.document import VectorStoreInfo

vectorstore_info = VSI(

    name="annual_report",
    description="a banking annual report as a pdf",
    vectorstore = db
)
# Create a vector store toolkit
toolkit = VST(vectorstore_info=vectorstore_info)

# Create a vector store agent
agent_executor = create_vectorstore_agent(llm, toolkit)

# # Create a vector store agent
# agent_executor = create_vectorstore_agent(llm, vectorstore_info)

In [ ]:

st.title('🦜🔗 GPT Investment Banker')
# Create a text input box for the user
prompt = st.text_input('Input your prompt here')

# If the user hits enter
if prompt:
    # Then pass the prompt to the LLM
    response = agent_executor.run(prompt)
    # ...and write it out to the screen
    st.write(response)

    # With a streamlit expander
    with st.expander('Document Similarity Search'):
        # Find the relevant pages
        search = store.similarity_search_with_score(prompt)
        # Write out the first
        st.write(search[0][0].page_content)

In [ ]:
!streamlit run "/content/drive/MyDrive/Colab Notebooks/gpt investor.ipynb"